In [1]:
from utils import setup_env_path

setup_env_path()

Former working directory:  c:\Users\corne\Documents\2-Scolarite\1-AgroParisTech\4-Cursus\3-3A\5-Cours\14-Fil_Rouge\fil-rouge-pollinisateurs\notebooks
Current working directory:  c:\Users\corne\Documents\2-Scolarite\1-AgroParisTech\4-Cursus\3-3A\5-Cours\14-Fil_Rouge\fil-rouge-pollinisateurs


In [27]:
import os
import pandas as pd
import warnings
from pytaxize.itis import hierarchy_full
from pytaxize.refactor import Refactor
from pytaxize import ncbi
from pytaxize import Ids

os.environ['ENTREZ_KEY'] = "8c5e06834ba59d32554e4ee00d59a3a6e008"

In [3]:
# Les fonctions ci-dessous devraient être accessibles directement avec pytaxize
# Mais il y a des problèmes lors de l'importation
# Les recopier ici résoud ce problème 

def hierarchy(ids):
    """
    Get a full taxonomic hierarchy from NCBI

    :param ids: one or more NCBI taxonomy ids

    :note: Remember to set your Entrez API key as `ENTREZ_KEY`

    :return: dict, named with ids given to `ids`, 
        where each value in the dict is a list of taxa, each
        a dict with the fields ``ScientificName``, ``Rank``, and ``TaxId``

    Usage::

        from pytaxize import ncbi
        ncbi.hierarchy(ids=9606)
        ncbi.hierarchy(ids=[9606,55062,4231])
    """
    toget = ["ScientificName", "Rank", "TaxId"]
    key = os.environ.get("ENTREZ_KEY")
    if key is None:
        raise Exception("ENTREZ_KEY is not defined")
    if not isinstance(ids, list):
        ids = [ids]
    idz = ",".join([str(x) for x in ids])
    args = {"db": "taxonomy", "ID": idz, "api_key": key}
    res = _entrez("efetch", args)
    taxa = res.xpath("//TaxaSet/Taxon")
    out = []
    for x in range(len(taxa)):
        nodes = taxa[x].xpath(".//LineageEx/Taxon")
        tmp = [
            dict(zip(toget, [node.xpath(w)[0].text for w in toget])) for node in nodes
        ]
        tmp.append(dict(zip(toget, [taxa[x].xpath(w)[0].text for w in toget])))
        out.append(tmp)
    return dict(zip(ids, out))


class Classification(object):
    """
    Classification: Retrieve taxonomic hierarchy for taxonomic IDs

    Usage::

        from pytaxize import Classification
        
        # ITIS
        ## one id
        x = Classification(99208)
        x
        x.ids
        res = x.itis()
        res[99208]
        
        ## many ids - with one invalid id
        x = Classification([99208, 129313, 9999999])
        x
        x.ids
        res = x.itis()
        res[99208]
        res[129313]
        res[9999999]

        # NCBI
        x = Classification(9606)
        x
        x.ids
        x.ncbi()
    """

    def __init__(self, ids):
        if isinstance(ids, int):
            ids = [ids]
        self.ids = ids

    def __repr__(self):
        x = """<%s>\n""" % type(self).__name__
        y = """  ids: %s""" % ",".join([str(w) for w in self.ids[:10]])
        return x + y

    def itis(self):
        out = []
        for i in range(len(self.ids)):
            id = self.ids[i]
            res = hierarchy_full(id)
            if res[0] is None:
                warnings.warn("No results for taxon '" + str(id) + "'")
                res = {}
            out.append(res)
        out = dict(zip(self.ids, out))
        return out

    def ncbi(self):
        res = hierarchy(self.ids)
        # out = []
        # for i in range(len(self.ids)):
        #     id = self.ids[i]
            # if res[0] is None:
            #     warnings.warn("No results for taxon '" + str(id) + "'")
            #     res = {}
            # out.append(res)
        # out = dict(zip(self.ids, out))
        return res

def _entrez(path="esearch", args={}):
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/%s.fcgi" % path
    tt = Refactor(url, args, request="get").xml()
    return tt

In [6]:
insects = pd.read_csv('data/insect/insect.csv')

In [7]:
insect_unknown = []
dictionary = {}

for insect in insects['insecte_sc']:
    
    # Cleaning of insects names
    insect_process = insect.replace(' et autres', '')          # on passe de 31 really unknown
    insect_process = insect_process.replace('plusieurs ', '')  # à ... avec ces deux commandes
    
    # Search if the insect is present in the ncbi database 
    search = ncbi.search(sci_com = insect_process)
    
    # If it is not present we put it in a list for ulterior treatment
    if len(search[insect_process]) == 0:
        insect_unknown.append(insect)
        print(search)
    
    # If it is present we search is classification
    else: 
        # Find the insect ID and have its classification
        id = int(search[insect_process][0]['TaxId'])
        x = Classification(id)
        x.ids
        try:
            classif = x.ncbi()
        except Exception as e:
            print("Une erreur s'est produite :", e)
        else:
            # Define the ranks we are interested in
            ranks = {'order': 'Ordre',
                     'superfamily': 'Super famille',
                     'family': 'Famille',
                     'subfamily': 'Sous famille',
                     'genus': 'Genre',
                     'species': 'Espèce'}
            # Extract the scientific names based on the rank
            names = {}
            for item in classif[id]:
                if item['Rank'] in ranks.keys():
                    names[ranks[item['Rank']]] = item['ScientificName']
            dictionary[insect] = names

{'cicadelles, cercopes': []}
Une erreur s'est produite : 400 Client Error: Bad Request for url: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=taxonomy&ID=1190796&api_key=8c5e06834ba59d32554e4ee00d59a3a6e008
Une erreur s'est produite : 400 Client Error: Bad Request for url: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=taxonomy&ID=7456&api_key=8c5e06834ba59d32554e4ee00d59a3a6e008
Une erreur s'est produite : 400 Client Error: Bad Request for url: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=taxonomy&ID=43838&api_key=8c5e06834ba59d32554e4ee00d59a3a6e008
{'Aglais io': []}
Une erreur s'est produite : 400 Client Error: Bad Request for url: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=taxonomy&ID=1352504&api_key=8c5e06834ba59d32554e4ee00d59a3a6e008
Une erreur s'est produite : 400 Client Error: Bad Request for url: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=taxonomy&ID=88591&api_key=8c5e06834ba59d32554e4ee00d59a3a6

In [8]:
insect_error = ["Andrena clarkella et autres", "Polistes", "Syrphinae",
                "Helophilus, Parhelophilus", "Sphecodes et autres", "Pompilidae", 
                "Amegilla et autres", "Ebaeus", "Malachius et autres", 
                "Ceratina", "Ampedus et autres", "Limenitis", "Hemipenthes morio", 
                "Trichius", "Acmaeodera et autres", "Odezia atrata", 
                "Andrena albopunctata", "Coptosoma scutellatum", "Maculinea",
                "Nymphalis antiopa", "Satyrium spini", "Euphydryas cynthia", "Pyrgus sidae"]

In [9]:
for insect in insect_error:
    
    # Cleaning of insects names
    insect_process = insect.replace(' et autres', '')          
    insect_process = insect_process.replace('plusieurs ', '')  
    
    # Search if the insect is present in the ncbi database 
    search = ncbi.search(sci_com = insect_process)
    
    # If it is not present we put it in a list for ulterior treatment
    if len(search[insect_process]) == 0:
        insect_unknown.append(insect)
        print(search)
    
    # If it is present we search is classification
    else: 
        # Find the insect ID and have its classification
        id = int(search[insect_process][0]['TaxId'])
        x = Classification(id)
        x.ids
        try:
            classif = x.ncbi()
        except Exception as e:
            print("Une erreur s'est produite :", e)
        else:
            # Define the ranks we are interested in
            ranks = {'order': 'Ordre',
                     'superfamily': 'Super famille',
                     'family': 'Famille',
                     'subfamily': 'Sous famille',
                     'genus': 'Genre',
                     'species': 'Espèce'}
            # Extract the scientific names based on the rank
            names = {}
            for item in classif[id]:
                if item['Rank'] in ranks.keys():
                    names[ranks[item['Rank']]] = item['ScientificName']
            dictionary[insect] = names

Une erreur s'est produite : 400 Client Error: Bad Request for url: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=taxonomy&ID=124410&api_key=8c5e06834ba59d32554e4ee00d59a3a6e008
Une erreur s'est produite : 400 Client Error: Bad Request for url: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=taxonomy&ID=186074&api_key=8c5e06834ba59d32554e4ee00d59a3a6e008


In [11]:
insect_error_still = ["Limenitis", "Acmaeodera et autres"]

In [12]:
for insect in insect_error_still:
    
    # Cleaning of insects names
    insect_process = insect.replace(' et autres', '')          
    insect_process = insect_process.replace('plusieurs ', '')  
    
    # Search if the insect is present in the ncbi database 
    search = ncbi.search(sci_com = insect_process)
    
    # If it is not present we put it in a list for ulterior treatment
    if len(search[insect_process]) == 0:
        insect_unknown.append(insect)
        print(search)
    
    # If it is present we search is classification
    else: 
        # Find the insect ID and have its classification
        id = int(search[insect_process][0]['TaxId'])
        x = Classification(id)
        x.ids
        try:
            classif = x.ncbi()
        except Exception as e:
            print("Une erreur s'est produite :", e)
        else:
            # Define the ranks we are interested in
            ranks = {'order': 'Ordre',
                     'superfamily': 'Super famille',
                     'family': 'Famille',
                     'subfamily': 'Sous famille',
                     'genus': 'Genre',
                     'species': 'Espèce'}
            # Extract the scientific names based on the rank
            names = {}
            for item in classif[id]:
                if item['Rank'] in ranks.keys():
                    names[ranks[item['Rank']]] = item['ScientificName']
            dictionary[insect] = names

In [19]:
print(len(dictionary))
print(len(insect_unknown)) # 2 times Dasytidae
print(len(insects['insecte_sc'].unique()))

507

In [22]:
df = pd.read_csv('data/insect/insect.csv')

In [24]:
with pd.option_context('mode.chained_assignment', None):
    # Création des colonnes Ordre, Super famille, Famille, Sous famille, Genre, Espèce
    df['classification'] = df['insecte_sc'].apply(lambda x: dictionary[x] if x in dictionary else None)

    df['Ordre'] = df['classification'].apply(lambda x: x['Ordre'] if isinstance(x, dict) and 'Ordre' in x else None)
    df['Super famille'] = df['classification'].apply(lambda x: x['Super famille'] if isinstance(x, dict) and 'Super famille' in x else None)
    df['Famille'] = df['classification'].apply(lambda x: x['Famille'] if isinstance(x, dict) and 'Famille' in x else None)
    df['Sous famille'] = df['classification'].apply(lambda x: x['Sous famille'] if isinstance(x, dict) and 'Sous famille' in x else None)
    df['Genre'] = df['classification'].apply(lambda x: x['Genre'] if isinstance(x, dict) and 'Genre' in x else None)
    df['Espèce'] = df['classification'].apply(lambda x: x['Espèce'] if isinstance(x, dict) and 'Espèce' in x else None)

    df.drop(columns=['classification'], inplace=True)

,insecte_sc,insecte_fr,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Apis mellifera,L'Abeille mellifère,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,None
2,Sarcophaga,Les Mouches à damier,Diptera,Oestroidea,Sarcophagidae,Sarcophaginae,Sarcophaga,None
3,Tettigonioidea,Les Sauterelles,Orthoptera,Tettigonioidea,None,None,None,None
4,"Neomyia, Calliphora et autres",Les Mouches aux reflets métalliques,Diptera,Oestroidea,Calliphoridae,Calliphorinae,Calliphora,None
5,Diptera,Les Mouches difficiles à déterminer,Diptera,None,None,None,None,None
6,Tephritidae et autres,Les Téphritides et autres,Diptera,Tephritoidea,Tephritidae,None,None,None
7,"Eupeodes, Scaeva",Les Syrphes à taches en virgules,Diptera,Syrphoidea,Syrphidae,Syrphinae,Scaeva,None
8,Episyrphus balteatus,Le Syrphe ceinturé,Diptera,Syrphoidea,Syrphidae,Syrphinae,Episyrphus,Episyrphus balteatus
9,Meliscaeva et autres,Les Syrphes à abdomen fin,Diptera,Syrphoidea,Syrphidae,Syrphinae,Meliscaeva,None


In [53]:
df.head()

,insecte_sc,insecte_fr,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Apis mellifera,L'Abeille mellifère,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,None
2,Sarcophaga,Les Mouches à damier,Diptera,Oestroidea,Sarcophagidae,Sarcophaginae,Sarcophaga,None
3,Tettigonioidea,Les Sauterelles,Orthoptera,Tettigonioidea,None,None,None,None
4,"Neomyia, Calliphora et autres",Les Mouches aux reflets métalliques,Diptera,Oestroidea,Calliphoridae,Calliphorinae,Calliphora,None


La liste des inconnus est assez courte. Les retrouver à la main n'est pas impossible si on a le temps.

In [26]:
insect_unknown

['cicadelles, cercopes et autres',
 'Aglais io',
 'Tibellus, Paratibellus',
 'plusieurs Dasytidae',
 'Polyommatus, Plebeius et autres',
 'Nomioides, Celhalictus',
 'plusieurs Dasytidae',
 'Bibio hortulanus',
 'Sapigidae',
 'Opsilia coerulescens',
 'Bombylella atra',
 'Glaphyra et autres',
 'Hoplia coerulea',
 'Hemipenthes maura',
 'Aricias nicias',
 'Hemipenthes velutina',
 'Usia',
 'Vadonia unipunctata',
 'Chlorophorus pilosus',
 'Eclimus gracilis',
 'Oberea erythrocephala']

In [31]:
df.to_csv('data/insect/insect_classif_unique.csv', index=False)

In [32]:
df_unique = pd.read_csv('data/insect/insect_classif_unique.csv')

In [33]:
# Import spipoll.csv as a pandas dataframe
spipoll = pd.read_csv("data/spipoll.csv", low_memory=False)

# Extract relevant columns from the spipoll dataframe
df_collection = spipoll[['collection_id', 'insecte_sc', 'insecte_fr']]
df_collection.shape # (670744, 3)

(670744, 3)

On doit faire la jointure sur insecte_fr car la colonne insecte_sc a été modifiée.

Pour rappel, des noms ont été ajoutés et une correction orthographique a été faite.

In [49]:
merged_df = pd.merge(df_collection, df_unique, on='insecte_fr', how='left')
merged_df.shape # (670744, 9)

(670744, 10)

In [50]:
merged_df.head(10)

,collection_id,insecte_sc_x,insecte_fr,insecte_sc_y,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,1,Apis mellifera,L'Abeille mellifère,Apis mellifera,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,1,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,Bombus,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,NaN
2,2,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,Bombus,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,NaN
3,2,Sarcophaga,Les Mouches à damier,Sarcophaga,Diptera,Oestroidea,Sarcophagidae,Sarcophaginae,Sarcophaga,NaN
4,2,Apis mellifera,L'Abeille mellifère,Apis mellifera,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
5,3,Tettigonoidea,Les Sauterelles,Tettigonioidea,Orthoptera,Tettigonioidea,NaN,NaN,NaN,NaN
6,4,"Neomyia, Calliphora et autres",Les Mouches aux reflets métalliques,"Neomyia, Calliphora et autres",Diptera,Oestroidea,Calliphoridae,Calliphorinae,Calliphora,NaN
7,4,NaN,Les Mouches difficiles à déterminer,Diptera,Diptera,NaN,NaN,NaN,NaN,NaN
8,4,Apis mellifera,L'Abeille mellifère,Apis mellifera,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
9,4,NaN,Les Mouches difficiles à déterminer,Diptera,Diptera,NaN,NaN,NaN,NaN,NaN


In [51]:
merged_df.rename(columns={'insecte_sc_x': 'insecte_sc'}, inplace=True)
merged_df.drop(columns=['insecte_sc_y'], inplace=True)

In [54]:
merged_df.head(10)

,collection_id,insecte_sc,insecte_fr,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,1,Apis mellifera,L'Abeille mellifère,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,1,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,NaN
2,2,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,NaN
3,2,Sarcophaga,Les Mouches à damier,Diptera,Oestroidea,Sarcophagidae,Sarcophaginae,Sarcophaga,NaN
4,2,Apis mellifera,L'Abeille mellifère,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
5,3,Tettigonoidea,Les Sauterelles,Orthoptera,Tettigonioidea,NaN,NaN,NaN,NaN
6,4,"Neomyia, Calliphora et autres",Les Mouches aux reflets métalliques,Diptera,Oestroidea,Calliphoridae,Calliphorinae,Calliphora,NaN
7,4,NaN,Les Mouches difficiles à déterminer,Diptera,NaN,NaN,NaN,NaN,NaN
8,4,Apis mellifera,L'Abeille mellifère,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
9,4,NaN,Les Mouches difficiles à déterminer,Diptera,NaN,NaN,NaN,NaN,NaN


In [52]:
merged_df.to_csv('data/insect/insect_classif.csv', index=False)